In [1]:
import utils
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import model as models
import gc

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
# 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 10)])
    except RuntimeError as e:
        print(e)

In [3]:
datafile_path = os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), 
                        ('data/server/')) 

In [4]:
print(datafile_path)

/data_disk/home/joongho/FL/data/server/


In [5]:
train_DS, val_DS = utils.data_loader(datafile_path)

Found 50000 files belonging to 10 classes.
Using 40000 files for training.
Found 50000 files belonging to 10 classes.
Using 10000 files for validation.


In [6]:
general_model = models.get_convnext_model(
    input_shape = (100, 100, 3),
    num_classes = 10
)

In [7]:
weight_save_path = os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), 
                                ('models/general/weights')) 

In [8]:
general_model.load_weights(weight_save_path)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()
monitor = 'val_accuracy'
general_model.compile(optimizer = optimizer, loss = loss_fn, metrics=['accuracy'])

In [9]:
general_model.evaluate(val_DS, verbose = 1)

 89/313 [=======>......................] - ETA: 27s - loss: 1.9337 - accuracy: 0.6591

KeyboardInterrupt: 

In [17]:
client_weights_path = list()
client_weights = list()
for i in range(3):
    client_weights_path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), 
                                ('models/clients/client{}/weights').format(i)))
    client_model = models.get_convnext_model(
                        input_shape = (100, 100, 3),
                        num_classes = 10
                    )
    client_weights.append(client_model.get_weights())

In [25]:
avg_weights = list()
for i, j, k in zip(client_weights[0], client_weights[1], client_weights[2]):
    avg_weights.append((i + j + k)/3)

In [26]:
client_model.set_weights(avg_weights)
client_model.compile(optimizer = optimizer, loss = loss_fn, metrics=['accuracy'])

In [27]:
client_model.evaluate(val_DS, verbose=1)

134/313 [===========>..................] - ETA: 40s - loss: 2.4478 - accuracy: 0.0991

KeyboardInterrupt: 